## 基于阿里云的 PAI 环境实验指导


## 热身

一键搭建 stable diffusion

![sd-step1](./img/sd-step1.png)
![sd-step2](./img/sd-step1.png)
![sd-step3](./img/sd-step3.png)

#### 等部署完成后，点击启动 Web 应用即可

#### 实验提示词：

```
A photorealistic, highly detailed illustration of Ana de Armas in a vintage Hollywood style, reminiscent of the golden age of cinema, with a focus on glamour and elegance.
```

![sd-web-ui](./img/sd-web-ui.png)


## 基于 chatglm2-6b 模型制作镜像

目前可以免费领取额度：https://free.aliyun.com/?pipCode=learn&spm=5176.14066474.J_5834642020.5.5abb4c5fJ1rOHH

当然也可以同样用我们已经训练好的模型

需要准备：

- Docker 环境
- 创建一个名字位：`llms-lab` 文件夹，作为项目工程目录
- 下载好 `chatglm2-6b` 模型，放在 `llms-lab` 文件夹内

后面操作都是在 `llms-lab` 文件夹内进行

### 第一步：制作基于模型的镜像

创建一个 `Dockerfile.llms` 文件，内容如下

```Dockerfile
FROM registry.cn-beijing.aliyuncs.com/agi/pai-base:v1

# WORKDIR 指定工作目录
WORKDIR /home/

# 安装包
COPY chatglm2-6b chatglm2-6b
```

#### 构建基于模型的镜像并且指定镜像名字为：llms:v1

执行下面命令

```
docker build -t llms:v1 . -f Dockerfile.llms
```

#### 验证结果

运行下面命令，会看到我们制作后的镜像列表

```
docker images
```


### 第二步：制作基于业务代码的镜像

在本地建立一个 `Dockerfile` 文件

准备好业务代码，本教程以 `openai-api.py` 文件当做业务代码，此文件放在了 lab 文件夹下面。

```Dockerfile
FROM llms:v1

# WORKDIR 指定工作目录
WORKDIR /home/

# 业务代码
COPY openai-api.py openai-api.py
COPY requirements.txt requirements.txt

# 安装项目依赖的包
RUN pip install -r requirements.txt


# EXPOSE 暴露 80 端口
EXPOSE 80
EXPOSE 7000
```

#### 构建基于业务代码的镜像

运行下面命令，来制作基于业务代码的镜像，名字为：agilab:v1

```
docker build -t agilab:v1 . -f Dockerfile
```


#### 构建基于业务代码的镜像并且指定镜像名字为：agilab:v1

```
docker build -t agilab:v1 . -f Dockerfile
```


### 第三步：把业务镜像推送到阿里云

方便后续阿里云 PAI 环境的 EAW 部署需要

<!-- 通过 `https://cr.console.aliyun.com/cn-beijing/instances` 进入阿里云容器镜像服务ACR -->

#### 阿里在阿里云上推送自己制作的镜像需要使用阿里云的容器镜像服务（Container Registry）。

1. **注册阿里云账号**：如果你还没有阿里云账号，首先需要注册一个。

2. **开通容器镜像服务**：登录到阿里云控制台，通过 `https://cr.console.aliyun.com/cn-beijing/instances` ，地区选择 `华北2（北京）` ，后续 `PAI` 地区要对应 , 找到 “容器镜像服务” 选择个人并开通。

3. **创建命名空间**：在容器镜像服务控制台中，创建一个新的命名空间。命名空间是用来组织镜像的，你可以根据项目或组织来创建。

4. **创建仓库**：在你的命名空间下，创建一个新的仓库。仓库是用来存储镜像的。

5. **登录阿里云 Docker Registry**：在你的本地机器上，使用 Docker CLI 登录到阿里云的 Docker Registry。你可以在容器镜像服务控制台中找到登录命令，通常是这样的：

   - 下面命令里面的 `star*****@163.com` 加密文本需要更改为自己的邮箱

   ```
   ## 登录阿里云 ACR
   docker login --username=star*****@163.com registry.cn-beijing.aliyuncs.com
   ```

6. **标记你的镜像**：在推送镜像之前，你需要先标记（tag）你的镜像，使其与阿里云的仓库匹配。

   在标记镜像前，我们需要执行`docker images` 列出我们想要的镜像 id

   ![镜像列表](./img/image-list-desc.png)

   注意：

   - a.下面命令里面的 `ImageId` 换成自己的镜像 ID
   - b.`镜像版本号` 修改为 `v1`
   - c.`agi/pai-base` 命名空间和镜像名字换成自己的

   ```bash
   docker tag [ImageId] registry.cn-beijing.aliyuncs.com/[命名空间]/[镜像名字]:[镜像版本号]
   ```

   参考案例：

   ```
   docker tag ce047f91539c registry.cn-beijing.aliyuncs.com/agi/pai-base:v1
   ```

7. **推送镜像**：使用以下命令将你的镜像推送到阿里云：

   上面制作好的镜像，使用以下命令推送到阿里云

   ```bash
   docker push registry.cn-beijing.aliyuncs.com/[命名空间]/[镜像名字]:[镜像版本号]
   ```

8. **验证**：返回到阿里云的容器镜像服务控制台，检查你的仓库，你应该能看到你刚刚推送的镜像。

   ![检查镜像](./img/check-image.png)


### 第四步：在 EAS 上运行我们制作的镜像

- 需要注意的是我们需要把 7000 端口映射出来

首先打开 EAS 控制台

![EAS 控制台](./img/eas-01.png)

然后点击服务部署,部署方式选择：镜像部署 AI+Web 应用

![部署方式](./img/build-way.png)

填写制作好的业务镜像、启动服务的脚本、端口。

- 切记端口不要填写：8080

![填写镜像地址](./img/deploy-edit-image.png)

选择服务器配置和磁盘扩容

- 选择试用活动，可以免费使用

![选择系统配置](./img/choice-config.png)

最后检查相关配置和部署

![检查和部署](./img/check-and-deploy.png)


### 部署成功的状态

![部署成功的状态](./img/deploy-success.png)

### 测试服务

参考下图，点击在测试，在后面输入下面的 url

```
/v1/chat/completions
```

在传参里面填写下面内容

```
{
  "messages": [
    {
      "role": "user",
      "content": "你是谁"
    }
  ],
  "stream": true,
  "model": "gpt-3.5-turbo",
  "temperature": 1,
  "presence_penalty": 0,
  "frequency_penalty": 0,
  "top_p": 1
}
```

![测试服务](./img/llms-test.png)

左侧界面显示返回的数据，恭喜你成功部署了自己的大模型服务。


#### 如果不想自己制作镜像，已经为大家准备好了

- 镜像地址：registry.cn-beijing.aliyuncs.com/itxishu/itxishu-base:v2
- 启动地址填写：python /home/lab/openai-api.py


### 让 EAS 可以使用外网域名访问 （选学）

因为 EAS 的外面服务单独调用需要加上 token，并且是一个阿里云的多级子域名，如果对外提供服务，一般需要专线映射到公网 ip 或者把提供的外网域名做一层转发映射到我们自己的域名。

后面的教程是带领大家，如何让阿里云 EAS 提供的域名，绑定到我们自己的域名并且对外提供服务。

#### 配置 NGINX 代理

- 由于需要外网的域名需要 token 我们自己做一层透传

```nginx
location /openai/ {
        proxy_pass http://v3.********.cn-beijing.pai-eas.aliyuncs.com/;
        proxy_set_header Authorization "MjU1ZjZiY****************A==";
        # 添加 CORS 头部
        if ($request_method = 'OPTIONS') {
            add_header 'Access-Control-Allow-Origin' *;
            #add_header 'Access-Control-Allow-Origin' 'https://chat.devagi.cn';
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS, PUT, DELETE';
            add_header 'Access-Control-Allow-Headers' 'Authorization, Content-Type, x-requested-with';  # 添加 x-requested-with
            add_header 'Access-Control-Max-Age' 1728000;
            add_header 'Content-Type' 'text/plain; charset=utf-8';
            add_header 'Content-Length' 0;
            return 204;
        }
        proxy_buffering off;

    }

```

### 完整案例

```nginx
server
    {
        listen 443 ssl http2;
        #listen [::]:443 ssl http2;
        server_name api.agiclass.ai ;
        index index.html index.htm default.html default.htm;
        ssl_certificate /home/wwwroot/shudong-api/nginx/ssl/api.shudong.wang.pem;
        ssl_certificate_key /home/wwwroot/shudong-api/nginx/ssl/api.shudong.wang.key;
        ssl_session_timeout 5m;
        ssl_protocols TLSv1 TLSv1.1 TLSv1.2 TLSv1.3;
        ssl_prefer_server_ciphers on;
        ssl_ciphers "TLS13-AES-256-GCM-SHA384:TLS13-CHACHA20-POLY1305-SHA256:TLS13-AES-128-GCM-SHA256:TLS13-AES-128-CCM-8-SHA256:TLS13-AES-128-CCM-SHA256:EECDH+CHACHA20:EECDH+CHACHA20-draft:EECDH+AES128:RSA+AES128:EECDH+AES256:RSA+AES256:EECDH+3DES:RSA+3DES:!MD5";
        ssl_session_cache builtin:1000 shared:SSL:10m;
        ssl_dhparam /usr/local/nginx/conf/ssl/dhparam.pem;

# 配置llms的代理
location /openai/ {
        proxy_pass http://v3.1526322695392938.cn-beijing.pai-eas.aliyuncs.com/;
        proxy_set_header Authorization "MjU1ZjZiY2VlODdmYTU2YzdmYjg0YWZjNjg5ODZkNGU5YzgzNDhhMA==";
	# 添加 CORS 头部
        if ($request_method = 'OPTIONS') {
            add_header 'Access-Control-Allow-Origin' *;
            #add_header 'Access-Control-Allow-Origin' 'https://chat.devagi.cn';
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS, PUT, DELETE';
            add_header 'Access-Control-Allow-Headers' 'Authorization, Content-Type, x-requested-with';  # 添加 x-requested-with
            add_header 'Access-Control-Max-Age' 1728000;
            add_header 'Content-Type' 'text/plain; charset=utf-8';
            add_header 'Content-Length' 0;
            return 204;
        }

    }
        access_log  /home/wwwlogs/api.agiclass.ai.log;
    }
```


### 结论

当访问：`https://api.agiclass.ai/openai/` 相当于访问 `http://v3.1526322695392938.cn-beijing.pai-eas.aliyuncs.com/`


## 通过配置，我们可以通过外网域名访问我们的 EAS 服务

### 使用开源 webUI 测试我们的大模型接口

- https://github.com/Yidadaa/ChatGPT-Next-Web

### 基于 ChatGPT-Next-Web 为大家部署好在线 WebUI 服务

通过：https://chat.devagi.cn 可以访问

点击左下角的齿轮 ⚙ 设置按钮后，填写我们已经反向代理后的接口 `https://api.agiclass/openai/` 即可

![webui配置](./img/webui-config.png)


### 在 WebUI 上测试

![chat ui](./img/chat-ui-x.png)
